# Transfer Attack
This notebook provides our transferability analysis results on the recovered facial images.

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from tqdm.auto import tqdm

from nbnet import get_nbnet
from feats.utils_feat import load_bin
from preset import get_transfer_preset
from eval_tools import eval_bbs   
from eval_attack import tf_attack_ours
device = "" # your device setting "cpu" or "cuda"

### Load Transfer target backbone

##### param_dir : transfer attack target model parameter's directory

#### Reference of pre-trained models
##### - R100_Glint_Cosface : [Insightface] https://github.com/deepinsight/insightface/tree/master/model_zoo 
##### - R50_Webface_Arcface : [Insightface] https://github.com/deepinsight/insightface/tree/master/model_zoo 
##### - SF20_VGGface_Sphereface : [Opensphere] https://github.com/ydwen/opensphere
##### - SF64_MS1MV2_Sphereface : [Opensphere] https://github.com/ydwen/opensphere
##### - Vit_MS1MV3_Cosface : https://github.com/zhongyy/Face-Transformer

In [ ]:
param_dir  = "" # your parmeter's directory, you can download pre-trained models from above links
backbones = get_transfer_preset(param_dir = param_dir, device = device)

### Load dataset

##### test_dir : testset's directory(LFW dataset)

In [ ]:
test_dir = ""  # input your dataset(LFW)'s directory
lfw = load_bin(test_dir + '/lfw.bin', (112,112))

### Evaluation of transfer taget models

##### backbones : Lists of transfer target models
##### lfw : lfw datasets
##### device : device

**NOTE** This reproduces Table 5.

In [ ]:
result = eval_bbs(backbones, lfw, device)

### Transfer attacks on target models

**NOTE** This reproduces Table 6.

In [ ]:
nbnet_dir = "" # input your pre-trainde NbNet model's directory
nbnet = get_nbnet(nbnet_dir, device)

In [ ]:
pre_dir = "" # directory where you've already saved the recovered pre-images from LSH templates.
dir_configs = dict()
dir_configs["BH"] = pre_dir + "BH.pt"
dir_configs["GRP"] = pre_dir + "GRP.pt"
dir_configs["URP"] = pre_dir + "URP.pt"
dir_configs["DIoM"] = pre_dir + "ABH.pt"
dir_configs["IMM"] = pre_dir + "IMM.pt"
dir_configs["ABH"] = pre_dir + "DIoM.pt"
dir_configs["Jiang"] = pre_dir + "Jiang.pt"

In [ ]:
lsh_names = ["BH", "GRP", "URP", "DIoM", "IMM", "ABH", "Jiang"]
for name in lsh_names:
    _ = tf_attack_ours(name, lfw, result, nbnet, dir_configs, device)